1. 성능평가지표 f1, AUC O
2. voting(soft) x
3. quantization o
4. cnn 계열 모델(densenet, inception-v3, resnet18, 34, 50) o
5. scheduler(reduceLRonPlateau) o
6. 증강 x
ppt
1.스팩트로그램 변환 o
2.split(val) o


grid-search
ablation=optuna

In [ ]:
# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 압축 해제 명령
!unzip -qq "/content/drive/MyDrive/dataset.zip" -d dataset

In [ ]:
# 압축 해제 명령
!unzip -qq "/content/drive/MyDrive/dataset1.zip" -d dataset1

Mounted at /content/drive


In [ ]:
!ls /content/dataset1/val_spectrograms/0

TRAIN_0019.png	TRAIN_0897.png	TRAIN_1687.png	TRAIN_2357.png	TRAIN_3346.png	TRAIN_4472.png
TRAIN_0061.png	TRAIN_0961.png	TRAIN_1737.png	TRAIN_2414.png	TRAIN_3357.png	TRAIN_4482.png
TRAIN_0092.png	TRAIN_1018.png	TRAIN_1759.png	TRAIN_2419.png	TRAIN_3452.png	TRAIN_4502.png
TRAIN_0095.png	TRAIN_1046.png	TRAIN_1797.png	TRAIN_2431.png	TRAIN_3569.png	TRAIN_4509.png
TRAIN_0260.png	TRAIN_1107.png	TRAIN_1813.png	TRAIN_2444.png	TRAIN_3576.png	TRAIN_4513.png
TRAIN_0315.png	TRAIN_1110.png	TRAIN_1814.png	TRAIN_2445.png	TRAIN_3633.png	TRAIN_4522.png
TRAIN_0322.png	TRAIN_1204.png	TRAIN_1827.png	TRAIN_2516.png	TRAIN_3648.png	TRAIN_4543.png
TRAIN_0442.png	TRAIN_1288.png	TRAIN_1839.png	TRAIN_2558.png	TRAIN_3652.png	TRAIN_4622.png
TRAIN_0617.png	TRAIN_1317.png	TRAIN_1985.png	TRAIN_2659.png	TRAIN_3710.png	TRAIN_4650.png
TRAIN_0657.png	TRAIN_1327.png	TRAIN_2010.png	TRAIN_2661.png	TRAIN_3764.png	TRAIN_4686.png
TRAIN_0682.png	TRAIN_1396.png	TRAIN_2015.png	TRAIN_2771.png	TRAIN_3792.png	TRAIN_4687.png
TRAIN_0740

In [ ]:
pip install librosa matplotlib

In [ ]:
pip install scikit-learn

In [ ]:
# torch 관련 라이브러리
from torchvision import models, transforms
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torchaudio
import torch.nn.functional as F
from torch.optim.lr_scheduler import MultiStepLR, StepLR
from torchsummary import summary
import torchaudio.transforms as T

# 일반 라이브러리
import argparse
import numpy as np
import random
import os
from PIL import Image
import matplotlib.pyplot as plt
import time
from tqdm import tqdm
from sklearn.metrics import f1_score

 #스팩트로그램
import librosa
import librosa.display
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
from PIL import Image

# 스케쥴러, AUC
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import models, transforms
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn.metrics import f1_score, roc_auc_score
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

In [ ]:
# 모델 학습 관련 파라미터 모음 --> 자유롭게 변경하고, 추가해보세요.

class Args():
  data_type = "2d"
  scheduler = "reducelronplateau"
  model = "resnet"
  n_class = 2
  epoch = 50
  phase = "train"
  model_path = "./model_weight_2d.pth"

args = Args()


In [ ]:
# 데이터 세트 관련 함수 --> 데이터 증강 기법을 적절하게 추가해보세요.
class TimeMasking(object):
    def __init__(self, T=40, max_masks=1):
        self.T = T
        self.max_masks = max_masks

    def __call__(self, spec):
        for _ in range(0, self.max_masks):
            t = random.randrange(0, self.T)
            t0 = random.randrange(0, spec.shape[1] - t)
            spec[:, t0:t0+t] = 0
        return spec

class FrequencyMasking(object):
    def __init__(self, F=30, max_masks=1):
        self.F = F
        self.max_masks = max_masks

    def __call__(self, spec):
        for _ in range(0, self.max_masks):
            f = random.randrange(0, self.F)
            max_f0 = spec.shape[0] - f
            if max_f0 <= 0:
                continue
            f0 = random.randrange(0, max_f0)
            spec[f0:f0+f, :] = 0
        return spec

class ImageDataset(Dataset):
    def __init__(self, directory, transform=None):
        self.directory = directory
        self.transform = transform
        self.classes = sorted(os.listdir(directory))
        self.class_to_idx = {cls_name: idx for idx, cls_name in enumerate(self.classes)}
        self.samples = []


        for class_name in self.classes:
            class_dir = os.path.join(directory, class_name)
            for image_name in os.listdir(class_dir):
                self.samples.append((os.path.join(class_dir, image_name), self.class_to_idx[class_name]))

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        image_path, label = self.samples[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
# 데이터 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 학습용 데이터셋 및 데이터 로더
train_dataset = ImageDataset('/content/dataset1/train_spectrograms', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

# 검증용 데이터셋 및 데이터 로더
val_dataset = ImageDataset('/content/dataset1/val_spectrograms', transform=transform)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
# 학습용 데이터셋의 크기 출력
print(f"학습 데이터 수는 {len(train_dataset)}개 입니다.")

# 검증용 데이터셋의 크기 출력
print(f"검증 데이터 수는 {len(val_dataset)}개 입니다.")

학습 데이터 수는 2800개 입니다.
검증 데이터 수는 704개 입니다.


In [ ]:
# 이미지 파일 경로
image_path = '/content/dataset/train_spectrograms/0/TRAIN_0007.png'

# 이미지 열기
image = Image.open(image_path)

# 이미지 시각화
plt.figure(figsize=(10, 10))
plt.imshow(image)
plt.axis('off')  # 축 표시 안함
plt.show()

In [ ]:
def get_model(name, n_class, pretrained=True): # pretrained는 항상 False로 두어야 합니다. True로 학습할 경우 감점(-5)
    if name == 'vgg16':
        model = models.vgg16(pretrained=pretrained)
        num_features = model.classifier[2].in_features
        model.classifier[2] = nn.Linear(num_features, n_class)
    elif name == 'resnet':
        model = models.resnet18(pretrained=pretrained)
        num_features = model.fc.in_features
        model.fc = nn.Linear(num_features, n_class)
    elif name == 'densenet':
        model = models.densenet121(pretrained=pretrained)
        num_features = model.classifier.in_features
        model.classifier = nn.Linear(num_features, n_class)
    elif name == 'inception_v3':
        model = models.inception_v3(pretrained=pretrained)
        # Replace the classifier with a fully connected layer with n_class outputs
        model.fc = nn.Linear(model.fc.in_features, n_class)
    return model

resnet = get_model('resnet', args.n_class)
vgg16 = get_model('vgg16', args.n_class)
densenet = get_model('densenet', args.n_class)
inception_v3 = get_model('inception_v3', args.n_class)

class Simple1DCNN(nn.Module):
    def __init__(self):
        super(Simple1DCNN, self).__init__()
        self.conv1 = nn.Conv1d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv1d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.adaptive_pool = nn.AdaptiveAvgPool1d(1000)  # 출력을 1000의 길이로 고정
        self.fc = nn.Linear(64 * 1000, 6)  # Adaptive Pooling 이후의 출력 크기에 맞춰 조정

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = self.adaptive_pool(x)  # Adaptive Pooling 적용
        x = x.view(-1, 64 * 1000)  # 적절한 크기로 플래트닝
        x = self.fc(x)
        return x


class Simple2DCNN(nn.Module):
    def __init__(self):
        super(Simple2DCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, padding=1)  # 입력 채널 3, 출력 채널 64
        self.conv2 = nn.Conv2d(64, 128, kernel_size=3, padding=1)  # 입력 채널 64, 출력 채널 128
        self.conv3 = nn.Conv2d(128, 256, kernel_size=3, padding=1)  # 입력 채널 128, 출력 채널 256
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)  # 2x2 풀링
        self.adaptive_pool = nn.AdaptiveAvgPool2d(7)
        self.fc = nn.Linear(256 * 7 * 7, 6)  # 최종 출력을 위한 완전 연결 계층

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # 첫 번째 컨볼루션 + 풀링
        x = self.pool(F.relu(self.conv2(x)))  # 두 번째 컨볼루션 + 풀링
        x = self.pool(F.relu(self.conv3(x)))  # 세 번째 컨볼루션 + 풀링
        x = self.adaptive_pool(x)  # Adaptive Pooling 적용
        x = x.view(-1, 256 * 7 * 7)  # 플래트닝
        x = self.fc(x)  # 완전 연결 계층
        return x

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.10/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


In [ ]:
# utils에 포함되었던 함수들 + tic toc 추가. tic toc은 수정하지마세요.
def visualize_audio_batch(audio_signals, labels):
    fig, axes = plt.subplots(4, 4, figsize=(10, 5))

    for i, ax in enumerate(axes.flat):
        if i >= 16:  # 16개의 오디오만 표시
            break
        ax.plot(audio_signals[i].t().numpy())  # 오디오 신호 플롯
        ax.set_title(f'Label: {labels[i]}')
        ax.axis('off')

    plt.tight_layout()
    plt.show()

def tic():
    # 현재 시간을 전역 변수에 저장
    global start_time
    start_time = time.time()

def toc():
    # tic()이 호출된 후 경과한 시간을 계산하고 출력
    elapsed_time = time.time() - start_time

    hours = int(elapsed_time // 3600)
    minutes = int((elapsed_time % 3600) // 60)
    seconds = elapsed_time % 60
    print(f"학습에 소요된 시간은 총 : {hours}시간 {minutes}분 {seconds}초 입니다.")

In [ ]:
def train_model(model, train_loader, val_loader, epochs, device, args):
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.004)

    if args.scheduler == 'multistep':
        scheduler = MultiStepLR(optimizer, milestones=[5, 10], gamma=0.1)
    elif args.scheduler == 'steplr':
        scheduler = StepLR(optimizer, step_size=5, gamma=0.1)
    elif args.scheduler == 'reducelronplateau':
        scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)

    for epoch in range(epochs):
        # Training
        model.train()
        running_loss = 0.0
        total_batches = len(train_loader)
        correct_predictions = 0
        total_samples = 0
        all_labels = []
        all_predictions = []
        all_outputs = []

        for i, (audio_signals, labels) in enumerate(tqdm(train_loader)):
            audio_signals, labels = audio_signals.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(audio_signals)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs, 1)
            correct_predictions += (predicted == labels).sum().item()
            total_samples += labels.size(0)

            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())
            all_outputs.extend(outputs.detach().cpu().numpy())  # Use detach() before numpy()

        avg_loss = running_loss / total_batches
        accuracy = correct_predictions / total_samples
        train_f1_score = f1_score(all_labels, all_predictions, average='weighted')

        try:
            train_auc = roc_auc_score(all_labels, all_outputs, multi_class='ovr')
        except ValueError:
            train_auc = None  # Handle the case when AUC cannot be computed

        print(f"Epoch [{epoch + 1}/{epochs}], Train Loss: {avg_loss:.4f}, Train Accuracy: {accuracy:.4f}, Train F1 Score: {train_f1_score:.4f}, Train AUC: {train_auc}")

        # Validation
        model.eval()
        val_loss = 0.0
        correct_val_predictions = 0
        total_val_samples = 0
        all_val_labels = []
        all_val_predictions = []
        all_val_outputs = []

        with torch.no_grad():
            for audio_signals, labels in tqdm(val_loader):
                audio_signals, labels = audio_signals.to(device), labels.to(device)

                outputs = model(audio_signals)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                correct_val_predictions += (predicted == labels).sum().item()
                total_val_samples += labels.size(0)

                all_val_labels.extend(labels.cpu().numpy())
                all_val_predictions.extend(predicted.cpu().numpy())
                all_val_outputs.extend(outputs.detach().cpu().numpy())  # Use detach() before numpy()

        avg_val_loss = val_loss / len(val_loader)
        val_accuracy = correct_val_predictions / total_val_samples
        val_f1_score = f1_score(all_val_labels, all_val_predictions, average='weighted')

        try:
            val_auc = roc_auc_score(all_val_labels, all_val_outputs, multi_class='ovr')
        except ValueError:
            val_auc = None  # Handle the case when AUC cannot be computed

        print(f"Epoch [{epoch + 1}/{epochs}], Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Val F1 Score: {val_f1_score:.4f}, Val AUC: {val_auc}")

        # Scheduler update
        if args.scheduler == 'reducelronplateau':
            scheduler.step(avg_val_loss)
        else:
            scheduler.step()

    print('Finished Training')

    # Save the model
    torch.save(model.state_dict(), f"./model_weight_{args.data_type}.pth")
    return model


In [ ]:
# 모델 불러오기
if args.data_type == "1d":
    model = Simple1DCNN()

elif args.data_type == "2d":
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        TimeMasking(T=40, max_masks=1),
        FrequencyMasking(F=30, max_masks=1)
    ])

    if args.model == 'vgg16':
        model = get_model(args.model, args.n_class, pretrained=False)
    elif args.model == 'resnet':
        model = get_model(args.model, args.n_class, pretrained=False)
        #quantized_model = torch.quantization.quantize_dynamic(
            #model, {nn.Conv2d, nn.Linear}, dtype=torch.qint8
        #)
        #quantized_model.eval()
    elif args.model == 'densenet':
        model = get_model(args.model, args.n_class, pretrained=False)
        quantized_model = torch.quantization.quantize_dynamic(
            model, {nn.Conv2d, nn.Linear}, dtype=torch.qint8
        )
        quantized_model.eval()
    elif args.model == 'inception_v3':
        model = get_model(args.model, args.n_class, pretrained=False)
        quantized_model = torch.quantization.quantize_dynamic(
            model, {nn.Conv2d, nn.Linear}, dtype=torch.qint8
        )
        quantized_model.eval()
    elif args.model == 'simple':
        model = Simple2DCNN()

    train_dataset = ImageDataset(directory='dataset1/train_spectrograms', transform=transform)
    val_dataset = ImageDataset(directory='dataset1/val_spectrograms', transform=transform)


# 데이터 로더 재생성
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=2, shuffle=False)

# Device 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
print(f"학습 데이터 수는 {len(train_dataset)}개 입니다.")
print(f"검증 데이터 수는 {len(val_dataset)}개 입니다.")

In [ ]:
# 모델 요약
if args.data_type == "1d":
    summary(model.cuda(), (1,16000))
elif args.data_type == "2d":
    summary(model.cuda(), (3,224,224))

In [ ]:
# 학습 시작 및 종료에 걸린 시간을 측정하기 위한 tic - toc

tic()

model = train_model(model, train_loader, val_loader, epochs=args.epoch, device=device, args=args)

toc()

In [ ]:
# 테스트 데이터에 대해 inference하기 위한 코드 입니다.
if args.data_type == "1d":
    test_dataset = AudioDataset(directory='/content/dataset/test')
elif args.data_type == "2d":
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ])
    test_dataset = ImageDataset(directory='/content/dataset/test_spectrograms', transform=transform)

test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print(f"테스트 데이터 수는 {len(test_dataset)}개 입니다.")

In [ ]:
def evaluate_model(model, test_loader, device, args):
    if args.model_path:
        model.load_state_dict(torch.load(args.model_path, map_location=device))
    model = model.to(device)

    model.eval()

    criterion = nn.CrossEntropyLoss()

    total = 0
    correct = 0
    total_loss = 0.0
    all_labels = []
    all_predictions = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

            # F1 score 계산을 위한 레이블과 예측 값 저장
            all_labels.extend(labels.cpu().numpy())
            all_predictions.extend(predicted.cpu().numpy())

    avg_loss = total_loss / len(test_loader)
    accuracy = correct / total * 100
    # F1 score 계산, average='macro'
    f1 = f1_score(all_labels, all_predictions, average='macro')

    print(f'Test Accuracy: {accuracy:.2f}%, Avg Loss: {avg_loss:.4f}, F1 Score: {f1:.4f}')

    return accuracy, avg_loss, f1

In [ ]:
# 모델 평가
accuracy, avg_loss, f1 = evaluate_model(model, test_loader, device=device, args=args)
print(f"테스트 데이터의 f1 score는 {f1}")